In [6]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.cluster import KMeans, OPTICS, SpectralClustering
import umap

# Read in dataset
daily = pd.read_csv('data/final_processed/daily_prices.csv', parse_dates=['date']).sort_values(['date','ticker']).set_index('date')
ratios = pd.read_csv('data/final_processed/firm_ratios.csv', parse_dates=['date']).sort_values(['date','ticker']).set_index('date')
sectors = pd.read_csv('data/final_processed/sectors.csv', parse_dates=['date']).sort_values(['date','ticker']).set_index('date')
short = pd.read_csv('data/final_processed/short_interest_rate.csv', parse_dates=['date']).sort_values(['date','ticker']).set_index('date')

# Merge
df = daily.merge(ratios, on=['ticker', 'date'])
df = df.merge(short, on =['ticker', 'date'])
df = df.merge(sectors, on=['ticker', 'date'])

# Define the date range of the dataset
start_date = '2002-01'
end_date = '2019-12'

# Create a date range with a customizable frequency
formation_period = '3M'
date_range = pd.date_range(start=start_date, end=end_date, freq=formation_period)

# Loop through all formation periods
results = []
for period_start in date_range:
    period_end = period_start + pd.tseries.offsets.DateOffset(months=int(formation_period[:-1]))
    df_period = df.loc[period_start.strftime('%Y-%m'):period_end.strftime('%Y-%m')]
    to_drop = df_period.loc[df_period.isna().any(axis=1)]['ticker'].unique()
    df_period = df_period.loc[~df_period['ticker'].isin(to_drop)]

    # Preprocessing steps
    df_train = df_period.reset_index().sort_values(['ticker', 'date'])
    idx = df_train[['ticker', 'date']]
    df_train = df_train.drop(['date','ticker'], axis=1)

    ohe_column = 'gicdesc'
    ohe_categories = df_train[ohe_column].unique().tolist()
    enc = OneHotEncoder(sparse_output=False, categories=[ohe_categories]) 
    transformer = make_column_transformer((enc, [ohe_column]), remainder='passthrough') 
    X_train = transformer.fit_transform(df_train)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    # Dimensionality reduction methods and clustering algorithms
    dim_reduction_methods = [
        {'name': 'PCA', 'method': PCA, 'params': {'n_components': [2, 3, 4]}},
        {'name': 'KPCA', 'method': KernelPCA, 'params': {'n_components': [2, 3, 4], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}},
        {'name': 'UMAP', 'method': umap.UMAP, 'params': {'n_components': [2, 3, 4], 'n_neighbors': [5, 10, 15]}}
    ]

    clustering_algorithms = [
        {'name': 'KMeans', 'method': KMeans, 'params': {'n_clusters': [2, 3, 4, 5]}},
        # {'name': 'OPTICS', 'method': OPTICS, 'params': {'min_samples': [3, 5, 7]}},
        {'name': 'SpectralClustering', 'method': SpectralClustering, 'params': {'n_clusters': [2, 3, 4, 5]}}
    ]

    # Loop through combinations of dimensionality reduction and clustering methods
    for dr in dim_reduction_methods:
        for cl in clustering_algorithms:
            for dr_param_value in dr['params']['n_components']:
                reduced_data = dr['method'](n_components=dr_param_value).fit_transform(X_train)

                # Create a new df with components
                components_df = pd.DataFrame(data=reduced_data, columns=[f'pc {i+1}' for i in range(dr_param_value)])

                # Merge the principal components with indices then groupby tickers
                merged_df = pd.concat([idx.reset_index(drop=True), components_df], axis=1)
                grouped_df = merged_df.groupby('ticker')

                # Concatenate the components for each ticker into a single vector
                vecs = {}
                for ticker, group in grouped_df:
                    components = group[[f'pc {i+1}' for i in range(dr_param_value)]].values.T
                    vec_components = np.concatenate(components)
                    vecs[ticker] = vec_components

                # Create df with a single row for each ticker and vectorized components
                vectorized_df = pd.DataFrame(list(vecs.items()), columns=['ticker', 'components'])

                # Get mode of the vector lengths
                vector_lengths = [len(vector) for vector in vectorized_df['components']]
                lengths_series = pd.Series(vector_lengths)
                mode_length = lengths_series.mode().iloc[0]

                # Drop anything that doesn't have the right length components; should do it by mode of vector lengths
                for i, vector in enumerate(vectorized_df['components']):
                    if vector.shape != (mode_length,):
                        vectorized_df = vectorized_df.drop(i)

                # Extract the vectors from the vectorized_df DataFrame
                vectors = np.array([np.array(vector) for vector in vectorized_df['components']])

                for param_name, param_values in cl['params'].items():
                    for param_value in param_values:
                        clustering_method = cl['method'](**{param_name: param_value})
                        cluster_labels = clustering_method.fit_predict(vectors)
                        # Merge the labels with the indices
                        labeled_df = pd.concat([vectorized_df['ticker'].reset_index(drop=True), pd.Series(cluster_labels, name='cluster')], axis=1)

                        # Group the tickers by the assigned cluster labels
                        clusters = labeled_df.groupby('cluster')['ticker'].apply(list)

                        # Store the results
                        result = {
                            'period_start': period_start,
                            'period_end': period_end,
                            'dim_reduction_method': dr['name'],
                            'dim_reduction_params': {f'n_components': dr_param_value},
                            'clustering_algorithm': cl['name'],
                            'clustering_params': {param_name: param_value},
                            'clusters': clusters
                        }
                        results.append(result)

# Print the results
for r in results:
    print(f"Period: {r['period_start']} - {r['period_end']}")
    print(f"Dimensionality Reduction: {r['dim_reduction_method']} - {r['dim_reduction_params']}")
    print(f"Clustering Algorithm: {r['clustering_algorithm']} - {r['clustering_params']}")
    print("Clusters:")
    print(r['clusters'])
    print("\n")



/var/folders/pr/c3wk8t5n3q14vnms13t5jjmh0000gn/T/ipykernel_57915/2065095237.py:11: DtypeWarning: Columns (70) have mixed types. Specify dtype option on import or set low_memory=False.
  ratios = pd.read_csv('data/final_processed/firm_ratios.csv', parse_dates=['date']).sort_values(['date','ticker']).set_index('date')
/Users/danielisaacgold/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/danielisaacgold/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/danielisaacgold/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' 